In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np

# Load model
model_id = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # mac lol
model = model.to(device)

In [74]:
import csv
import ast
import os
import sys

data = []
DATA_DIR = "../data/"

csv.field_size_limit(sys.maxsize)

for file in os.listdir(DATA_DIR):
  print(file)
  if file == "streetwear.csv":
    continue
  with open(DATA_DIR + file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    data.extend([row for row in reader])

# with open("../data/fashionhot.csv", "r", newline="", encoding="utf-8") as f:
#   reader = csv.DictReader(f, delimiter="\t")
#   data = [row for row in reader]

for i, row in enumerate(data):
  if row["Comments"] == "No comments" or row["Images"] == "No images":
    del data[i]
    continue
  data[i]["Comments"] = ast.literal_eval(data[i]["Comments"]) # maybe we find another way of storing the comments
print(len(data))

fashioncontroversial.csv
malefashioncontroversial.csv
fashionhot.csv
malefashionhot.csv
fashiontopyear.csv
mensfashiontopyear.csv
mensfashioncontroversialcontroversial.csv
streetweartopyear.csv
streetwearcontroversial.csv
streetwear.csv
OUTFITScontroversial.csv
malefashiontopyear.csv
OUTFITShot.csv
OUTFITStopyear.csv
mensfashionhot.csv
streetwearhot.csv
12559


In [40]:
len(data)

12464

In [ ]:
#getting comments sentiment analysis score

#1. Importing model
#used a huggingface sentiment analysis multilingual transformer. 
#Selected this one because of fine-grained classification of sentiments
#is multilingual distilbert fine-tuned into a sentiment analysis model
from transformers import pipeline
# Load the classification pipeline with the specified model
pipe = pipeline(model="tabularisai/multilingual-sentiment-analysis")
#into 512 tokens max
#Number of Classes: 5 (Very Negative, Negative, Neutral, Positive, Very Positive)
#Score: model's confidence in its result


[{'label': 'Neutral', 'score': 0.3992540240287781},
 {'label': 'Neutral', 'score': 0.5025282502174377},
 {'label': 'Neutral', 'score': 0.3811638057231903}]

In [4]:
import requests

def call_api(comments_list):
  r = requests.post(
    "http://192.168.158.88:8088/predict-sentiment/",
    json={
      "texts": comments_list
    }
  )
  return r.json()

call_api(["test", "test2", "test3"])

[{'text': 'test', 'label': 'Neutral', 'score': 0.39925417304039},
 {'text': 'test2', 'label': 'Neutral', 'score': 0.5025281310081482},
 {'text': 'test3', 'label': 'Neutral', 'score': 0.3811637759208679}]

In [75]:
#2 getting sentiment scores from individual comments in row[Comments]
# preparing output
sentiment_list = [] #list of dict:{average label: real from 1-5, average score: }
sentiment_mapping = {
    1: "Very Negative",
    2: "Negative",
    3: "Neutral",
    4: "Positive",
    5: "Very Positive"
}
numerical_mapping = {v: k for k, v in sentiment_mapping.items()}
progress = 0
#processing

from tqdm import tqdm


from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.environ.get("MONGODB_URI")

client = MongoClient(uri, server_api=ServerApi('1'))
db = client["fit-check"]
collection = db["embeddings"]

for row in tqdm(data[6787:]):
    try: # i give up
        total_weightage = 0
        total_result = 0
        comments_list = []
        score_list = []
        if row["Comments"] == ["No comments"]:
            continue
        for comment in row['Comments']:
            truncated_comment = comment['body']
            comments_list.append(truncated_comment)
            score_list.append(comment['score'])
        # result = pipe(comments_list, truncation=True, max_length=512)
        # print(result)
        result = call_api(comments_list)
        for i in range(len(comments_list)):
            label = numerical_mapping[result[i]['label']]
            weighted_result = score_list[i]*label
            total_weightage += score_list[i]
            total_result += weighted_result
        if total_weightage == 0:
            total_weightage = 1 #????
        post_score = total_result/total_weightage
        sentiment_list.append(post_score)
        #print(progress, end=" ")
        progress = progress+1

        collection.insert_one({
            "title": row["Title"],
            "comments": row["Comments"],
            "image": row["Images"],
            "score": row["Score"],
            "sentiment": post_score
        })
    except Exception as e:
        print(e)

        

  0%|          | 11/5772 [00:01<07:53, 12.16it/s]

string indices must be integers


  5%|▍         | 276/5772 [00:27<08:34, 10.69it/s]

string indices must be integers


  6%|▌         | 319/5772 [00:31<07:37, 11.93it/s]

...

100%|█████████▉| 5758/5772 [19:15<00:00, 15.65it/s]

string indices must be integers


100%|█████████▉| 5768/5772 [19:16<00:00, 17.68it/s]

string indices must be integers
string indices must be integers


100%|██████████| 5772/5772 [19:16<00:00,  4.99it/s]


In [ ]:
collection.aggregate([
    {"$group" : { "_id": "Title", "count": { "$sum": 1 } } },
    {"$match": {"_id" :{ "$ne" : 0 } , "count" : {"$gt": 1} } }, 
    {"$project": {"Title" : "$_id", "_id" : 0} }
]);


In [43]:
row

{'Title': 'GTA inspired outfits ',
 'Description': 'Mask: HUF\nHoodie: Uniqlo \nJersey: NHL\nTrousers: Dickies 874\nBoots: ROCA Supplies',
 'Score': '117',
 'URL': 'https://www.reddit.com/gallery/1h1r9ik',
 'Author': 'Oceani_MAOwg',
 'Created UTC': '2024-11-28 08:11:41',
 'Images': 'https://preview.redd.it/7vngmnq0pl3e1.jpg?width=3622&format=pjpg&auto=webp&s=ea33163cc3245181434aeb17d0041e6a0d7948f4, https://preview.redd.it/2pj6goq0pl3e1.jpg?width=4000&format=pjpg&auto=webp&s=dcd79c6c229741d689481055742e9f73a47afdad, https://preview.redd.it/36wo2oq0pl3e1.jpg?width=4000&format=pjpg&auto=webp&s=7a15a06fcf4688ff8326e7f10299b44479af8fdf, https://preview.redd.it/e3yg6nq0pl3e1.jpg?width=4000&format=pjpg&auto=webp&s=5a1f162971e833d9375215cc16f4376fad835643, https://preview.redd.it/hdprtnq0pl3e1.jpg?width=4000&format=pjpg&auto=webp&s=c5436798236dd324b12a4396cc8f37e3d1c90496, https://preview.redd.it/81zi0oq0pl3e1.jpg?width=4000&format=pjpg&auto=webp&s=2d20f95f5694754d5662681b92a06f2733f3bae5',
 

In [ ]:
#embedding
def run_embedding(images, titles, comments=None):
    """
    Takes in a list of PIL images and a list of texts (length <77 tokens each)
    Returns the image and text embedding vectors for the batch
    """
    inputs = processor(
        text=titles,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=77
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    image_embeds = torch.nn.functional.normalize(image_embeds)
    text_embeds = torch.nn.functional.normalize(text_embeds)

    return image_embeds, text_embeds


def combine_embeddings(image_embeds, text_embeds):
    return 0.5 * image_embeds + 0.5 * text_embeds # weighted average

In [7]:
# test mongo connection
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.environ.get("MONGODB_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
  client.admin.command('ping')
  print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
  print(e)

client.close()

Pinged your deployment. You successfully connected to MongoDB!


In [21]:
import requests
from PIL import Image
from io import BytesIO

from tqdm import tqdm
import gc

# embedded = []

client = MongoClient(uri, server_api=ServerApi('1'))
db = client["fit-check"]
collection = db["embeddings"]

batch_size = 16 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

for batch_rows in tqdm(list(batch(data, batch_size))):
    images = []
    titles = []
    comments_list = []
    image_urls = []
    scores = []
    for row in batch_rows:
        image_url = row["Images"].split(",")[0]
        print(image_url)
        if image_url == ["No images"]:
            continue
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        images.append(image)
        titles.append(row["Title"])
        # You can also batch comments if you want, or just use titles
        comments_list.append([row["Description"]] + [comment["body"] for comment in row["Comments"]])
        image_urls.append(image_url)
        scores.append(row["Score"])

    # Run batch embedding (titles only for simplicity)
    image_embeds, text_embeds = run_embedding(images, titles, None)  # Modify run_embedding to accept lists

    image_embeds_cpu = image_embeds.cpu()
    text_embeds_cpu = text_embeds.cpu()
    
    for idx in range(len(images)):
        collection.insert_one({
            "title": titles[idx],
            "comments": comments_list[idx],
            "image": image_urls[idx],
            "score": scores[idx],
            "image_embeds": {
                "type": "denseVector",
                "values": image_embeds_cpu[idx].tolist(),
            },
            "text_embeds": {
                "type": "denseVector",
                "values": text_embeds_cpu[idx].tolist(),
            },
            "embeds": {
                "type": "denseVector",
                "values": combine_embeddings(image_embeds_cpu[idx], text_embeds_cpu[idx]).tolist(),
            },
            "sentiment": sentiment_list[idx]
        })
        images[idx].close()
    del image_embeds, text_embeds
    gc.collect()

  0%|          | 0/62 [00:00<?, ?it/s]

https://preview.redd.it/wuldlke19tvd1.jpg?width=2316&format=pjpg&auto=webp&s=540e1aacf9c17e36ee23b358f1500b4e1db43d81
https://preview.redd.it/vhe7qe73vxdd1.jpg?width=1178&format=pjpg&auto=webp&s=a75dd488c7de94b6dd9d2af250d4c551d31316ff
https://i.redd.it/phtr7z19xz8d1.jpeg
https://preview.redd.it/49wnbjp25bvd1.jpg?width=1179&format=pjpg&auto=webp&s=c85ac5013f2a197d8c1145cb227f55ff8c496ace
https://preview.redd.it/5dryq4q8l8pd1.jpg?width=2080&format=pjpg&auto=webp&s=6d570e43c33db1106bfd41f03130ede84f6775cf
https://i.redd.it/ei7o5urb1rud1.jpeg
https://i.redd.it/ce6rr4bkvcad1.jpeg
https://preview.redd.it/r2d9ftruw14e1.jpg?width=960&format=pjpg&auto=webp&s=0970bc3a9d4af09e5f009dbcb2a3624116e4050d
https://preview.redd.it/vqkdt1pduypd1.jpg?width=3369&format=pjpg&auto=webp&s=73d2abe53c5f42ed26613c52c15b7d3cc78d1f10
https://preview.redd.it/gnosr651iuqd1.jpg?width=2160&format=pjpg&auto=webp&s=e66cb34c77d8247f91c06ca06de7cf3dab7ee4fb
https://preview.redd.it/mohphxtxlq0e1.jpg?width=761&format=pjpg&

  2%|▏         | 1/62 [00:04<04:21,  4.28s/it]

https://preview.redd.it/dlf7sbfed97e1.jpg?width=2791&format=pjpg&auto=webp&s=1d476d950a23c797c946831a82bdd6674131c0d3
https://preview.redd.it/vxvocpo1k23e1.png?width=1440&format=png&auto=webp&s=8e6859864310f15c57a4092d2710c57634156fed
https://preview.redd.it/9ki1bzeygp5e1.jpg?width=3024&format=pjpg&auto=webp&s=19034ec605f68f3bc32bc900cd0c6bf17a86aa21
https://preview.redd.it/iyj0xe4apu7e1.jpg?width=3072&format=pjpg&auto=webp&s=f6e905da6be4b525c5107aae4f1576a6b44cd0a1
https://preview.redd.it/5ssalpmng6wd1.jpg?width=3024&format=pjpg&auto=webp&s=02a9795a8722660cd7321b5f9b8787d99a6176c9
https://preview.redd.it/lpzvqjhajl8d1.jpg?width=1728&format=pjpg&auto=webp&s=a28b730a853eb793494909429843d3f2a5102eb0
https://preview.redd.it/gh5h8ne4up4d1.jpg?width=1440&format=pjpg&auto=webp&s=48776732a880d2fcffc3e1e12fb585aa2d6dd927
https://i.redd.it/ebg4xo4kn0od1.jpeg
https://preview.redd.it/a229ihn2vctd1.jpg?width=900&format=pjpg&auto=webp&s=5cbf5e2443231db0f11e8c4eb219f72195aec31b
https://i.redd.it/7hc

  3%|▎         | 2/62 [00:08<03:59,  3.99s/it]

...

 18%|█▊        | 11/62 [00:42<03:05,  3.64s/it]

https://preview.redd.it/29bojga99l5d1.jpg?width=2046&format=pjpg&auto=webp&s=add5d66182911ccd5c7e07196863bcddc3e61d55
https://i.redd.it/fk9mxc73z27e1.jpeg
https://preview.redd.it/t1jm99328q5e1.jpg?width=936&format=pjpg&auto=webp&s=55b4a74a5db76572def2eba73acb1f1f9d89aa8a
https://preview.redd.it/xow4plx3ukvd1.jpg?width=1079&format=pjpg&auto=webp&s=73d73aff1bca8a9a180d6f46d8d5a1e9c3719456
https://preview.redd.it/om81qmw6si6d1.jpg?width=1440&format=pjpg&auto=webp&s=240c33d0cc858c424145c923310dd6695f74ae16
https://i.redd.it/5bhvigwxvv1e1.jpeg
https://i.redd.it/8mtx0sfmnrsd1.jpeg
https://preview.redd.it/ux7xwjvlnh3d1.jpg?width=3024&format=pjpg&auto=webp&s=6f32799ebd77e13db394c00396f8da4d1f4bf22d
https://preview.redd.it/y2ney5k4bv8e1.jpg?width=3060&format=pjpg&auto=webp&s=85dfd53dbe15e0c19e63dfc45bed264e79a16374
https://i.redd.it/rdhs4rauoyvd1.jpeg
https://preview.redd.it/o6v96p2s4gxc1.jpg?width=2160&format=pjpg&auto=webp&s=a00432ca174c2b30b3730cd0c94db01f3fb3d673
https://i.redd.it/8pndkxw1x

 18%|█▊        | 11/62 [00:43<03:22,  3.97s/it]

['No images']


InvalidURL: Failed to parse: ['No images']

thank you gemini for the batching code

In [ ]:
# delete dupes

# Step 1: Aggregate duplicate titles with _ids
pipeline = [
    {
        "$group": {
            "_id": "$title",
            "ids": {"$push": "$_id"},
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}
        }
    }
]

duplicates = list(collection.aggregate(pipeline))

total_to_delete = 0
for doc in duplicates:
    title = doc["_id"]
    ids = doc["ids"]
    count = doc["count"]
    ids_to_delete = ids[1:]  # keep one, delete the rest

    print(f"Title: {title}")
    print(f"Occurrences: {count}")
    print(f"Deleting {len(ids_to_delete)} documents with _ids: {ids_to_delete}\n")

    # Uncomment this to delete:
    # collection.delete_many({"_id": {"$in": ids_to_delete}})
    total_to_delete += len(ids_to_delete)

print(f"Total duplicates marked for deletion: {total_to_delete}")


Total duplicates marked for deletion: 0


In [14]:
# Prepare inputs (image + optional text)
compare_text = "blue shirt and brown long pants"
compare_inputs = processor(
    text=[compare_text],
    return_tensors="pt",
    padding=True
)
compare_inputs = {k: v.to(device) for k, v in compare_inputs.items()} 

# Forward pass
with torch.no_grad():
    compare_text_embeds = model.get_text_features(**compare_inputs)

compare_text_embeds = torch.nn.functional.normalize(compare_text_embeds)

In [17]:
similarity = torch.nn.functional.cosine_similarity(image_embeds, text_embeds)
print(f"Image to post.txt similarity: {similarity[0]}")


similarity = torch.nn.functional.cosine_similarity(image_embeds, compare_text_embeds)
print(f"Image to manual description similarity: {similarity[0]}")

similarity = torch.nn.functional.cosine_similarity(text_embeds, compare_text_embeds)
print(f"Manual description to post.txt similarity: {similarity[0]}")

Image to post.txt similarity: 0.2717254161834717
Image to manual description similarity: 0.3059011697769165
Manual description to post.txt similarity: 0.4487333595752716


# previous code (no ai here)

In [ ]:
def run_embedding(image, title, comments):
    """
    Takes in a PIL image and some text (length <77 tokens)
    Returns the image and text embedding vectors separately
    """
    inputs = processor(
        # text=[title, *comments],
        text=[title],
        images=image,
        return_tensors="pt",
        padding=True,
        truncation=True # hm
    )
    inputs = {k: v.to(device) for k, v in inputs.items()} 

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    image_embeds = torch.nn.functional.normalize(image_embeds)
    text_embeds = torch.nn.functional.normalize(text_embeds)

    return image_embeds, text_embeds

def combine_embeddings(image_embeds, text_embeds):
    return 0.5 * image_embeds + 0.5 * text_embeds # weighted average

In [ ]:
# Run embeddings

import requests
from PIL import Image
from io import BytesIO

from tqdm import tqdm
import gc

embedded = []

# ...existing code...
max_tokens = 77

def truncate_text(text):
    tokens = processor.tokenizer(text, return_tensors="pt")
    if tokens['input_ids'].shape[1] > max_tokens:
        # Truncate to max_tokens
        return processor.tokenizer.decode(tokens['input_ids'][0][:max_tokens])
    return text

i = 0
for row in tqdm(data):
  image_url = row["Images"].split(",")[0] # D:
  if image_url == "No images":
    continue
  response = requests.get(image_url)
  image = Image.open(BytesIO(response.content))

  title = truncate_text(row["Title"])
  comments = [truncate_text(row["Description"])] + [truncate_text(comment["body"]) for comment in row["Comments"]]

  image_embeds, text_embeds = run_embedding(image, title, comments)
  image_embeds_cpu = image_embeds.cpu()
  text_embeds_cpu = text_embeds.cpu()

  del image_embeds, text_embeds
  if i % 20 == 19: gc.collect()
  image.close()

  embedded.append({
    "title": title,
    "comments": comments,
    "image": image_url,
    "score": row["Score"],
    "image_embeds": image_embeds_cpu,
    "text_embeds": text_embeds_cpu,
    "embeds": combine_embeddings(image_embeds_cpu, text_embeds_cpu)
  })
  i += 1

In [ ]:
count = 0
for row in data:
  text = row["Title"]
  tokens = processor.tokenizer(text, return_tensors="pt")
  token_count = tokens['input_ids'].shape[1]  # should be ≤ 77

  if token_count > 77:
    # print(token_count, text)
    count += 1

  for comment in row["Comments"]:
    text = comment["body"]
    tokens = processor.tokenizer(text, return_tensors="pt")
    token_count = tokens['input_ids'].shape[1]  # should be ≤ 77

    if token_count > 77:
      # print(token_count, text)
      count += 1

print(count)
# checking title length
